In [ ]:
import pandas as pd
from datetime import datetime as dt

import plotly
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots
from plotly.offline import iplot

import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)
cf.set_config_file(world_readable=True, theme='space', offline=True)

parser = lambda date: dt.strptime(date, '%d/%m/%y')

data = pd.read_csv("../input/covid19-in-india/covid_19_india.csv", index_col="Sno")
vac_data = pd.read_csv("../input/covid19-in-india/covid_vaccine_statewise.csv", index_col="Updated On", parse_dates=True, dayfirst=True)

data = data.rename(columns={"State/UnionTerritory": "Region"})

# India Covid-19 Situation

## Country

In [ ]:
country = data.groupby("Date").sum()
country.iplot(title="Country Situation")

# Increase In Cases feature
country["Increase In Cases"] = (country.Confirmed - country.Cured - country.Deaths)
country["Increase In Cases"].iplot(title="Increase In Cases (IIC)")

country["Cured/Confirmed"] = country.Cured / country.Confirmed
country["Deaths/Cured"] = country.Deaths / country.Cured

country[["Cured/Confirmed", "Deaths/Cured"]].iplot(title="Important Coefficients", )

## Regions

In [ ]:
regions = data.groupby("Region").sum()
regions["IIC"] = regions.Confirmed - regions.Cured - regions.Deaths
regions.sort_values(by="IIC", ascending=False, inplace=True)
regions.head(10)["IIC"].iplot(kind="bar", title="The 10 Greatest IIC in Regions")

regions_info = regions.reset_index()

fig_iic = px.choropleth(
    regions_info,
    geojson="https://gist.githubusercontent.com/jbrobst/56c13bbbf9d97d187fea01ca62ea5112/raw/e388c4cae20aa53cb5090210a42ebb9b765c0a36/india_states.geojson",
    featureidkey='properties.ST_NM',
    locations="Region",
    color="IIC",
    color_continuous_scale='Reds',
    title="IIC in Regions on the Map",
    template="plotly_dark"
)

fig_iic.update_geos(fitbounds="locations", visible=False)

fig_iic.show()

# India Covid-19 Vaccination Process

## Country

In [ ]:
vac_country = vac_data[vac_data.State == "India"].drop(columns=["State"]).dropna()
vac_country["Total Individuals Vaccinated"].iplot(title="Vaccination Process")

vac_country.rename(columns = {"Total Sputnik V Administered": "Sputnik V",
                            "Total Covaxin Administered": "Covaxin",
                            "Total CoviShield Administered": "CoviShield"},
                   inplace=True)

vac_country[["Sputnik V", "Covaxin", "CoviShield"]].iplot(title="Vaccine Types")

newest_country = vac_country.tail(1).T
newest_country.rename(index={"18-45 years (Age)": "18-45","45-60 years (Age)": "45-60","60+ years (Age)": "60+",
                             "Male(Individuals Vaccinated)": "Male", "Female(Individuals Vaccinated)": "Female"},
                      inplace=True)

newest_country.loc[["18-45", "45-60", "60+"], :].iplot(kind="bar", title="Age Distribution")
newest_country.loc[["Male", "Female"], :].iplot(kind="bar", title="Gender Distribution")

## Regions

In [ ]:
vac_regions = vac_data[vac_data.State != "India"].groupby("State").last() \
    .rename(columns={"Total Doses Administered": "Total Vaccinated"})

vac_regions.sort_values(by="Total Vaccinated", ascending=False, inplace=True)
vac_regions.head(10)["Total Vaccinated"].iplot(kind="bar", title="Top 10 Regions")

vac_regions_info = vac_regions.reset_index()

fig_states_vac = px.choropleth(
    vac_regions_info,
    geojson="https://gist.githubusercontent.com/jbrobst/56c13bbbf9d97d187fea01ca62ea5112/raw/e388c4cae20aa53cb5090210a42ebb9b765c0a36/india_states.geojson",
    featureidkey='properties.ST_NM',
    locations="State",
    color="Total Vaccinated",
    color_continuous_scale='Greens',
    title="Total Doses in Regions on the Map",
    template="plotly_dark"
)

fig_states_vac.update_geos(fitbounds="locations", visible=False)

fig_states_vac.show()

